In [1]:
# Threading
# To start Julia with 8 threads on PC, open anaconda prompt and run <set JULIA_NUM_THREADS=8>, then run <jupyter notebook>
# ENV["JULIA_NUM_THREADS"] = 8 # Don't need to run on mac - set nthreads in VScode
Threads.nthreads()

8

In [2]:
# import Pkg; Pkg.develop(path="/Users/evanheberlein/Library/CloudStorage/Box-Box/Cornell/IRQIV.jl")
using IRQIV
import PyPlot as plt
import Statistics, NaNStatistics
plt.pygui(true) # sets plots to popup instead of inline

true

In [3]:
# On PC 7/4/24: 
# re-installed IRQIV to desktop since Box installation not syncing to PC, ran Pkg.develop() on new path
# installed same Julia version as on Mac (1.9.3), installed IJulia to sync kernel to Jupyter

In [4]:
# Import images using IRQIV

FLIR_filename = joinpath(raw"/Volumes/ETH_4TB/CNRD_IR/Rec-DeFreesLab_-003504.ats")
# FLIR_filename = joinpath(raw"D:\CNRD_IR\Rec-DeFreesLab_-003504.ats")
Img_dims = [784, 1344]
DtFrames = 1
nImagePairs = Inf

# set range to average over and load images
inds = range(1, step=1, length=500)
raw_imgs = LoadATSImageSequence(FLIR_filename, inds, 2022)

nomean_imgs = zeros(size(raw_imgs))
for img = 1:size(raw_imgs, 3)
    nomean_imgs[:,:,img] = raw_imgs[:,:,img] .- Statistics.mean(raw_imgs[:,:,img])
end

In [5]:
# Visualize frames - SKIP
# Create a plot window and display the initial image
# fig, ax = plt.subplots()
# im = ax.imshow(raw_imgs[1], cmap="viridis", vmin=10000, vmax=10100)

# # Function to update the image
# function update_image!(im, new_img)
#     plt.imshow(new_img, cmap="viridis", vmin=10000, vmax=11000)
#     plt.draw()          # Force an update
#     plt.pause(0.001)
# end

# # Close all existing plot windows (optional, if needed)
# plt.close("all")

# # Update the image in a loop
# for i in 1:length(inds)
#     img = raw_imgs[i]
#     blur_img = zeros(size(raw_imgs[1]))
#     for n = 2:Img_dims[1]-1 # vertical dimension
#         for m = 2:Img_dims[2]-1 # horizontal dimension
#             blur_img[n,m] = Statistics.sum(img[n-1:n+1,m-1:m+1].*1/9)
#         end
#     end
#     update_image!(im, blur_img)
# end

# plt.show()

In [6]:
# Pixel-wise std
img_std = dropdims(Statistics.std(nomean_imgs[:,:,:], dims = 3), dims = 3)

# blur_std = zeros(size(img_std))
# for n = 2:Img_dims[1]-1 # vertical dimension
#     for m = 2:Img_dims[2]-1 # horizontal dimension
#         blur_std[n,m] = Statistics.sum(img_std[n-1:n+1,m-1:m+1].*1/9)
#     end
# end

plt.imshow(img_std)#, vmin=0, vmax=100)
plt.show()

In [7]:
# Edge detection from std dev image
import Images, ImageEdgeDetection
using ImageEdgeDetection: Percentile
filter_rad = 10
alg = ImageEdgeDetection.Canny(spatial_scale=filter_rad, high=Percentile(80), low=Percentile(20))
edges = ImageEdgeDetection.detect_edges(img_std, alg)
# mosaicview(img_std, edges; nrow=1)

# Convert a Float64 matrix to a boolean matrix based on a threshold
function float_to_bool(edges::Matrix{Float64}, threshold::Float64)
    return edges .> threshold
end

# Example usage
edges_bool = float_to_bool(edges, 0.5)

testlinerange = [150:300,50:400]
plt.imshow(edges_bool)#[testlinerange[1],testlinerange[2]])
plt.title("spatial scale = $filter_rad")
plt.show()

In [18]:
import Images
import PyPlot as plt

# Custom function to label components with 8-connectivity
function label_components_8_connected(edges::AbstractArray{Bool})
    rows, cols = size(edges)
    labels = zeros(Int, rows, cols)
    current_label = 0
    
    function in_bounds(r, c)
        return 1 <= r <= rows && 1 <= c <= cols
    end
    
    function dfs(r, c, label)
        stack = [(r, c)]
        while !isempty(stack)
            cr, cc = pop!(stack)
            if in_bounds(cr, cc) && edges[cr, cc] && labels[cr, cc] == 0
                labels[cr, cc] = label
                # Add all 8 neighbors
                for dr in -1:1, dc in -1:1
                    if !(dr == 0 && dc == 0)  # skip the current pixel itself
                        push!(stack, (cr + dr, cc + dc))
                    end
                end
            end
        end
    end
    
    for r in 1:rows
        for c in 1:cols
            if edges[r, c] && labels[r, c] == 0
                current_label += 1
                dfs(r, c, current_label)
            end
        end
    end
    
    return labels, current_label
end

# Example usage
# edges = rand(Bool, 256, 256)  # Example: using a random Bool matrix for demonstration
labels, num_components = label_components_8_connected(edges_bool)

# Plot the labeled components
plt.imshow(labels, cmap="nipy_spectral")
plt.colorbar(label="Component Label")
plt.title("Labeled Components with 8-Connectivity")

# Define the diagonal line by two points
ds_shadow_start = [1260, 1]  # [u, v]
ds_shadow_end = [1090, 784]  # [u, v]
plt.plot([ds_shadow_start[1], ds_shadow_end[1]], [ds_shadow_start[2], ds_shadow_end[2]], label="Diagonal Line", linewidth=2.5)
plt.show()


In [12]:
# Generate vector of labeled edges

import ImageMorphology
import Colors

# Assume edges is the boolean matrix from Canny edge detection

function extract_streamline_coords(edges::AbstractArray{Bool})
    # Label connected components
    labels, num_components = label_components_8_connected(edges)
    
    all_streamline_coords = Vector{Vector{Tuple{Int, Int}}}(undef, num_components)
    
    # Iterate over each component and extract coordinates
    for component in 1:num_components
        streamline_coords = Vector{Tuple{Int, Int}}()
        for i in CartesianIndices(edges)
            if labels[i] == component
                push!(streamline_coords, (i[2], i[1])) # j, i
            end
        end
        all_streamline_coords[component] = streamline_coords
    end
    
    return all_streamline_coords
end

# Example usage
all_streamline_coords_canny = extract_streamline_coords(edges_bool)

# all_streamline_coords now contains vectors of coordinates for each continuous line of 1's


784×1344 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 ⋮              ⋮              ⋮        ⋱           ⋮              ⋮        
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0   

In [13]:
# Function definitions - line identification

# Define the meshgrid function
function meshgrid(x::AbstractVector, y::AbstractVector)
    X = repeat(x', length(y), 1)
    Y = repeat(y, 1, length(x))
    return X, Y
end

# Bresenham's line algorithm to get pixels for a line
function bresenham_line(x0, y0, x1, y1)
    pixels = []
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    sx = x0 < x1 ? 1 : -1
    sy = y0 < y1 ? 1 : -1
    err = dx - dy

    while true
        push!(pixels, (x0, y0))
        if x0 == x1 && y0 == y1
            break
        end
        e2 = 2 * err
        if e2 > -dy
            err -= dy
            x0 += sx
        end
        if e2 < dx
            err += dx
            y0 += sy
        end
    end
    return pixels
end

# Wu's antialiasing algorithm - returns a raster along line w/ varying pixel intensities (smoothing effect)
function wu_line(x0, y0, x1, y1)
    function plot(x, y, c)
        return (x, y, c)
    end

    function ipart(x)
        return floor(Int, x)
    end

    function round(x)
        return ipart(x + 0.5)
    end

    function fpart(x)
        return x - floor(x)
    end

    function rfpart(x)
        return 1 - fpart(x)
    end

    steep = abs(y1 - y0) > abs(x1 - x0)

    if steep
        x0, y0 = y0, x0
        x1, y1 = y1, x1
    end

    if x0 > x1
        x0, x1 = x1, x0
        y0, y1 = y1, y0
    end

    dx = x1 - x0
    dy = y1 - y0
    gradient = dy / dx

    xend = round(x0)
    yend = y0 + gradient * (xend - x0)
    xgap = rfpart(x0 + 0.5)
    xpxl1 = xend
    ypxl1 = ipart(yend)
    pixels = [
        plot(xpxl1, ypxl1, rfpart(yend) * xgap),
        plot(xpxl1, ypxl1 + 1, fpart(yend) * xgap)
    ]

    intery = yend + gradient

    xend = round(x1)
    yend = y1 + gradient * (xend - x1)
    xgap = fpart(x1 + 0.5)
    xpxl2 = xend
    ypxl2 = ipart(yend)
    push!(pixels, plot(xpxl2, ypxl2, rfpart(yend) * xgap))
    push!(pixels, plot(xpxl2, ypxl2 + 1, fpart(yend) * xgap))

    for x in (xpxl1 + 1):(xpxl2 - 1)
        push!(pixels, plot(x, ipart(intery), rfpart(intery)))
        push!(pixels, plot(x, ipart(intery) + 1, fpart(intery)))
        intery += gradient
    end

    if steep
        pixels = [(y, x, c) for (x, y, c) in pixels]
    end

    return pixels
end

wu_line (generic function with 1 method)

In [14]:
import LinearAlgebra
import PyPlot as plt


# Function to check if a streamline intersects the diagonal line
function check_intersections(ds_shadow_start, ds_shadow_end, all_streamline_coords, length_threshold)
    # Call bresenham_line to generate pixel coordinates along the diagonal line
    line_coords = bresenham_line(ds_shadow_start[1], ds_shadow_start[2], ds_shadow_end[1], ds_shadow_end[2])
    
    # Initialize a vector to hold the intersecting streamlines
    intersecting_streamlines = []
    
    # Loop through each set of streamline coordinates
    for streamline_coords in all_streamline_coords
        # Calculate the length of the streamline
        streamline_length = LinearAlgebra.norm(streamline_coords[end] .- streamline_coords[1])
        
        # Only check streamlines that exceed the length threshold
        if streamline_length >= length_threshold
            # Check if any of the streamline coordinates intersect with the diagonal line
            for line_pix in line_coords
                if line_pix in streamline_coords
                    push!(intersecting_streamlines, streamline_coords)
                    break  # Stop checking this streamline if an intersection is found
                end
            end
        end
    end
    
    return intersecting_streamlines
end


# Set a length threshold (e.g., 100 pixels)
length_threshold = 200

# Find streamlines that intersect with the diagonal line
intersecting_streamlines = check_intersections(ds_shadow_start, ds_shadow_end, all_streamline_coords_canny, length_threshold)
intersecting_streamlines

# Optional: Plot the diagonal line and the intersecting streamlines
fig, ax = plt.subplots()

# Plot the diagonal line
ax.plot([ds_shadow_start[1], ds_shadow_end[1]], [ds_shadow_start[2], ds_shadow_end[2]], "r-", label="Diagonal Line")

# Plot all streamlines
for streamline_coords in all_streamline_coords_canny
    x_coords = [coord[1] for coord in streamline_coords]
    y_coords = [coord[2] for coord in streamline_coords]
    
    ax.plot(x_coords, y_coords, "b-", label="Streamline")
    ax.invert_yaxis()
end

# Highlight intersecting streamlines
for streamline_coords in intersecting_streamlines
    x_coords = [coord[1] for coord in streamline_coords]
    y_coords = [coord[2] for coord in streamline_coords]
    
    ax.plot(x_coords, y_coords, "g-", label="Intersecting Streamline", linewidth=2.5)
    ax.invert_yaxis()
end

plt.show()

In [15]:
# STIV test with variance streamline - manually select component
test_edge_pix = all_streamline_coords_canny[42] # Long streamline from edge of image to just before entrance
STIV_vec_Canny = zeros(length(test_edge_pix), length(raw_imgs))

# Create STIV image
for img = 1:length(raw_imgs)
    for coord = 1:length(test_edge_pix)
        j, i = test_edge_pix[coord]
        STIV_vec_Canny[coord, img] = raw_imgs[i, j, img]
    end
end

STIV_edges = ImageEdgeDetection.detect_edges(STIV_vec_Canny, alg)
STIV_edges = float_to_bool(STIV_edges, 0.5)
STIV_labels, num_components = label_components_8_connected(STIV_edges)

fig, (ax1, ax2) = plt.subplots(1, 2)
im1 = ax1.imshow(STIV_vec_Canny)
ax1.set_title("STIV image")
im2 = ax2.imshow(STIV_labels, cmap="nipy_spectral")
ax2.set_title("Labeled slopes")
fig.colorbar(im2, ax=ax2, label="Component Label")
plt.show()


In [16]:
length_threshold = 200

# Function to calculate the slope of each labeled component if it exceeds the length threshold
function calculate_slope_for_components(labels::AbstractArray{Int}, num_components::Int; length_threshold::Int=100)
    slopes = Dict{Int, Float64}()
    
    for label in 1:num_components
        # Find the coordinates of all pixels in the component
        component_coords = findall(x -> x == label, labels)
        
        # Only consider the component if it exceeds the length threshold
        if length(component_coords) > length_threshold
            # Find the first and last pixel coordinates in the component
            first_pixel = component_coords[1]
            last_pixel = component_coords[end]
            
            # Calculate the slope between the first and last pixel
            x1, y1 = first_pixel[2], first_pixel[1]  # (x1, y1)
            x2, y2 = last_pixel[2], last_pixel[1]    # (x2, y2)
            
            if x2 != x1
                slope = (y2 - y1) / (x2 - x1)
            else
                slope = Inf  # Vertical line (undefined slope)
            end
            
            # Store the slope in a dictionary
            slopes[label] = slope
        end
    end
    
    return slopes
end

# Example usage with a length threshold of 100 pixels
STIV_slopes = calculate_slope_for_components(STIV_labels, num_components, length_threshold=100)

# Print out the slopes of each component
for (label, slope) in STIV_slopes
    println("Component $label: Slope = $slope")
end

STIV_mean = Statistics.mean(values(STIV_slopes))


Component 5: Slope = 14.32
Component 56: Slope = 18.08823529411765
Component 30: Slope = 11.476190476190476
Component 67: Slope = 29.666666666666668
Component 45: Slope = 12.0
Component 112: Slope = 21.88235294117647
Component 64: Slope = 22.0
Component 104: Slope = 25.8125
Component 122: Slope = 24.166666666666668
Component 28: Slope = 14.363636363636363
Component 111: Slope = 30.0
Component 41: Slope = 19.823529411764707
Component 11: Slope = 10.2
Component 36: Slope = 17.6
Component 98: Slope = 21.857142857142858
Component 118: Slope = 23.4
Component 7: Slope = 12.818181818181818
Component 25: Slope = 13.0
Component 95: Slope = 21.181818181818183
Component 126: Slope = 26.25
Component 50: Slope = 28.75
Component 27: Slope = 16.790697674418606
Component 26: Slope = 34.0
Component 100: Slope = 24.944444444444443
Component 20: Slope = 13.583333333333334
Component 19: Slope = 13.285714285714286
Component 9: Slope = 13.5
Component 61: Slope = 19.61904761904762
Component 57: Slope = 13.12

19.990216672841584

In [19]:
# Loop through all intersecting streamlines and calculate STIV slope

# Preallocate for storing slopes
STIV_averages = zeros(length(intersecting_streamlines))
loop = 1

for streamline_coords in intersecting_streamlines
    # Create STIV vector for each streamline across all images
    STIV_vec = zeros(length(streamline_coords), size(nomean_imgs, 3))

    # Extract pixel values for the streamline across the image time series
    for img = 1:size(nomean_imgs, 3)
        for coord = 1:length(streamline_coords)
            j, i = streamline_coords[coord]
            STIV_vec[coord, img] = nomean_imgs[i, j, img] # i, j (row, col) should be correct order
        end
    end

    # Perform edge detection on the STIV image
    STIV_edges = ImageEdgeDetection.detect_edges(STIV_vec, alg)
    STIV_edges = float_to_bool(STIV_edges, 0.5)

    # Label connected components in the detected edges
    STIV_labels, num_components = label_components_8_connected(STIV_edges)

    # Calculate slope for components over a length threshold (e.g. 100 pixels)
    STIV_slopes = calculate_slope_for_components(STIV_labels, num_components, length_threshold=100)
    STIV_averages[loop] = Statistics.mean(values(STIV_slopes)) # Stored as a dictionary

    # Plot the results
    x_coords = [coord[1] for coord in streamline_coords]  # x = col index
    y_coords = [coord[2] for coord in streamline_coords]  # y = row index
    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

    # Plot the original image with the streamline
    im1 = ax1.imshow(img_std, cmap="viridis")  # Show the first frame or another reference image
    ax1.plot(x_coords, y_coords, "r", linewidth=2.5)  # Overlay the streamline in red
    ax1.set_title("Streamline")
    
    # Plot the STIV image
    im2 = ax2.imshow(STIV_vec, cmap="viridis")  # Display STIV vector
    ax2.set_title("STIV Image")
    
    # Plot the labeled components (slope map)
    im3 = ax3.imshow(STIV_labels, cmap="nipy_spectral")
    ax3.set_title("Labeled Slopes")
    fig.colorbar(im3, ax=ax3, label="Component Label")

    fig.suptitle("STIV Average Slope: $(STIV_averages[loop])", fontsize=14)

    loop += 1
end

plt.show()  # Display the plot


In [ ]:
# STIV test on single edge in subwindow of image 
# select area of edge image
edge_subarea = edges[testlinerange[1],testlinerange[2]]
edge_subarea .= ifelse.(edge_subarea .== 0.0, NaN, edge_subarea) # 0's to NaNs
stiv_length = length(testlinerange[2])
# create empty STIV vector
stiv_vec = zeros(stiv_length, length(raw_imgs))
# stiv_vec = fill(nan,(stiv_length, length(raw_imgs))) # dims: length of STIV line (rows) x number of images (cols)
for i = 1:length(raw_imgs)
    raw_img_subarea = raw_imgs[i][testlinerange[1],testlinerange[2]]
    raw_img_edge = raw_img_subarea.*edge_subarea # mask raw image by edge output NaN/1
    stiv_vec[:,i] = NaNStatistics.nanmean(raw_img_edge, dims = 1)
end

plt.imshow(stiv_vec)
plt.show()


In [ ]:
# Anti-aliasing Canny line - seems too complicated

import Images
import ImageEdgeDetection
import Colors
import LinearAlgebra
import PyPlot as plt

# Define the function to draw Wu's anti-aliased lines
function wu_line!(img, x0, y0, x1, y1)
    function ipart(x) return floor(Int, x) end
    function round_(x) return floor(Int, x + 0.5) end
    function fpart(x) return x - floor(x) end
    function rfpart(x) return 1.0 - fpart(x) end
    function plot(x, y, c)
        if 1 <= x <= size(img, 2) && 1 <= y <= size(img, 1)
            img[y, x] += c
        end
    end

    steep = abs(y1 - y0) > abs(x1 - x0)
    if steep
        x0, y0 = y0, x0
        x1, y1 = y1, x1
    end
    if x0 > x1
        x0, x1 = x1, x0
        y0, y1 = y1, y0
    end

    dx = x1 - x0
    dy = y1 - y0
    gradient = dy / dx

    xend = round_(x0)
    yend = y0 + gradient * (xend - x0)
    xgap = rfpart(x0 + 0.5)
    xpxl1 = xend
    ypxl1 = ipart(yend)
    if steep
        plot(xpxl1, ypxl1, rfpart(yend) * xgap)
        plot(xpxl1, ypxl1 + 1, fpart(yend) * xgap)
    else
        plot(ypxl1, xpxl1, rfpart(yend) * xgap)
        plot(ypxl1 + 1, xpxl1, fpart(yend) * xgap)
    end
    intery = yend + gradient

    xend = round_(x1)
    yend = y1 + gradient * (xend - x1)
    xgap = fpart(x1 + 0.5)
    xpxl2 = xend
    ypxl2 = ipart(yend)
    if steep
        plot(xpxl2, ypxl2, rfpart(yend) * xgap)
        plot(xpxl2, ypxl2 + 1, fpart(yend) * xgap)
    else
        plot(ypxl2, xpxl2, rfpart(yend) * xgap)
        plot(ypxl2 + 1, xpxl2, fpart(yend) * xgap)
    end

    if steep
        for x in (xpxl1 + 1):(xpxl2 - 1)
            y = ipart(intery)
            if !isnan(y)
                plot(x, y, rfpart(intery))
                plot(x, y + 1, fpart(intery))
            end
            intery += gradient
        end
    else
        for x in (xpxl1 + 1):(xpxl2 - 1)
            y = ipart(intery)
            if !isnan(y)
                plot(y, x, rfpart(intery))
                plot(y + 1, x, fpart(intery))
            end
            intery += gradient
        end
    end
end

# Function to convert Float64 matrix to Bool matrix based on a threshold
function float_to_bool(edges::Matrix{Float64}, threshold::Float64)
    return edges .> threshold
end

# Function to fit splines to detected edges
function fit_splines(edges::AbstractArray{Bool})
    # Placeholder function: In practice, use a library or algorithm to fit splines to the edges
    # This is a simplified example to illustrate the concept
    rows, cols = size(edges)
    splines = []
    
    for x in 1:cols
        for y in 1:rows
            if edges[y, x]
                push!(splines, (x, y))  # Collect points belonging to edges
            end
        end
    end

    return splines
end

# Function to draw splines using Wu's anti-aliased lines
function draw_splines(edges::AbstractArray{Bool}, threshold::Float64=0.5)
    edges_bool = float_to_bool(edges, threshold)
    splines = fit_splines(edges_bool)
    img_splines = zeros(Float64, size(edges))
    
    # Draw splines (or lines) using Wu's anti-aliased lines
    for i in 1:(length(splines) - 1)
        x0, y0 = splines[i]
        x1, y1 = splines[i + 1]
        wu_line!(img_splines, x0, y0, x1, y1)
    end

    # Normalize the img_splines matrix to scale the max value to 1
    img_splines ./= maximum(img_splines)
    
    return img_splines
end

# Example usage with an existing edges array
edges = rand(Float64, 256, 256)  # Example: using a random Float64 matrix
threshold = 0.5  # Define an appropriate threshold value
detected_splines_img = draw_splines(edges, threshold)  # Drawing splines from the edges matrix

# # Display the detected splines image
# plt.imshow(detected_splines_img, cmap="gray")
# plt.show()


In [22]:
# UPSTREAM TEST - VARIANCE STREAMLINE

FLIR_filename = joinpath(raw"/Volumes/ETH_4TB/CNRD_IR/Rec-DeFreesLab_-003365.ats")
raw_imgs = LoadATSImageSequence(FLIR_filename, inds, 2022) # Same parameters as downstream (500 frames)

# Pixel-wise std
img_std = dropdims(Statistics.std(raw_imgs[:,:,:], dims = 3), dims = 3)
blur_std = zeros(size(img_std))
for n = 2:Img_dims[1]-1 # vertical dimension
    for m = 2:Img_dims[2]-1 # horizontal dimension
        blur_std[n,m] = Statistics.sum(img_std[n-1:n+1,m-1:m+1].*1/9)
    end
end

plt.figure()
plt.imshow(img_std)#, vmin=0, vmax=100)

# Edge detection from std dev image
filter_rad = 10
alg = ImageEdgeDetection.Canny(spatial_scale=filter_rad, high=Percentile(80), low=Percentile(20))
edges = ImageEdgeDetection.detect_edges(img_std, alg)

# Example usage
threshold = 0.5  # Define an appropriate threshold value
edges_bool = float_to_bool(edges, threshold)

plt.figure()
plt.imshow(edges_bool)#[testlinerange[1],testlinerange[2]])
plt.title("spatial scale = $filter_rad")

edges_bool = float_to_bool(edges, threshold)
labels, num_components = label_components_8_connected(edges_bool)

# Plot the labeled components
plt.figure()
plt.imshow(labels, cmap="nipy_spectral")
plt.colorbar(label="Component Label")
plt.title("Labeled Components with 8-Connectivity")

all_streamline_coords_canny = extract_streamline_coords(edges_bool)

test_edge_pix = all_streamline_coords_canny[92] # Long streamline from edge of image to just before entrance
STIV_vec_Canny = zeros(length(test_edge_pix), length(raw_imgs))

for img = 1:length(raw_imgs)
    for coord = 1:length(test_edge_pix)
        i, j = test_edge_pix[coord]
        STIV_vec_Canny[coord, img] = raw_imgs[i, j, img]
    end
end

plt.figure()
plt.imshow(STIV_vec_Canny)
plt.show()


In [33]:
#### STREAMLINE METHOD 2 - MQD VECTOR FIELD

# Modifying downstream IRQIV code for same file as variance image

# Subwindow dimensions remain constant
const M=80 # subwindow dims - horiz, N = vert.
const N=80
const rhoI = 80 # how far away to search (+/-) - i=short side, j= long side
const rhoJ = 80 # Set these to visual estimates - plot one pair
MQD_parameters = (M=M, N=N, rhoI=rhoI, rhoJ=rhoJ)

# Set ATS parameters
Img_dims = [784, 1344] # Image dims: 784 x 1344
DtFrames = 1                    # Δt, frames (ATS 3116 is at 10Hz)
data_output_rate_frames = 1    # difference between I1 sequence, frames - time sep of output
nImagePairs = 100 # can set to inf - all frames in file
Nfiles = 40 # Set number of files to average over

# Load IR images:
I1inds = range(1, step=data_output_rate_frames, length=nImagePairs)
I2inds = I1inds .+ DtFrames

subwincorners = CartesianIndices(( (N+rhoI):N:(Img_dims[1]-(N+rhoI+1)), 
                                    (M+rhoJ):M:(Img_dims[2]-(M+rhoJ+1))))

filenum = 3504
FLIR_filename = joinpath("/Volumes/ETH_4TB/CNRD_IR/Rec-DeFreesLab_-00" * string.(filenum) * ".ats")
ats_year = 2022
@assert isfile(FLIR_filename)
I1images = LoadATSImageSequence(FLIR_filename, I1inds, ats_year)
I2images = LoadATSImageSequence(FLIR_filename, I2inds, ats_year)

# Remove image pairs when at least one image took place during NUC (calibration)
# mode:
if any(isnuc(I1images.Headers)) || any(isnuc(I2images.Headers))
    nuc_frames = isnuc(I1images.Headers) .| isnuc(I2images.Headers)
    I1inds = I1inds[.!( isnuc(I1images.Headers) .| isnuc(I2images.Headers) )]
    I2inds = I2inds[.!( isnuc(I1images.Headers) .| isnuc(I2images.Headers) )]
    I1images = LoadATSImageSequence(FLIR_filename, I1inds, ats_year)
    I2images = LoadATSImageSequence(FLIR_filename, I2inds, ats_year)
end

# Initialize MQD calculation:
MQD_variables = initialize_MQD(I1images,
                            I2images,
                            subwincorners,
                            N, M,
                            rhoI, rhoJ)                              

# Perform the MQD analysis:
subpix_shifts_i, subpix_shifts_j, performance_message = perform_MQD(
    I1images, I2images, subwincorners, MQD_variables)

# Plotting in pixel space (generate indices for vector locations) - issue w/ CSV package in multithread mode
x_vector = [index[2] for index in subwincorners]
y_vector = [index[1] for index in subwincorners]

plt.close("all")
fig,axs = plt.subplots()
strm = axs.streamplot(x_vector, y_vector, subpix_shifts_j[:,:,1], subpix_shifts_i[:,:,1], start_points = [(1120, 160)])
axs.quiver(x_vector, y_vector, subpix_shifts_j[:,:,1], subpix_shifts_i[:,:,1], angles="xy")
axs.invert_yaxis()
plt.show()



1.0%┣▍                                         ┫ 1/100 [00:00<Inf:Inf, InfGs/it]
9.0%┣████▏                                         ┫ 9/100 [00:01<00:07, 13it/s]
10.0%┣████▌                                        ┫ 10/100 [00:02<00:21, 4it/s]
16.0%┣███████▏                                     ┫ 16/100 [00:02<00:12, 7it/s]
17.0%┣███████▋                                     ┫ 17/100 [00:02<00:12, 7it/s]
18.0%┣████████                                     ┫ 18/100 [00:03<00:14, 6it/s]
19.0%┣████████▌                                    ┫ 19/100 [00:03<00:14, 6it/s]
23.0%┣██████████▍                                  ┫ 23/100 [00:03<00:11, 7it/s]
25.0%┣███████████▎                                 ┫ 25/100 [00:03<00:11, 7it/s]
27.0%┣████████████▏                                ┫ 27/100 [00:04<00:11, 7it/s]
28.0%┣████████████▋                                ┫ 28/100 [00:06<00:15, 5it/s]
35.0%┣███████████████▊                             ┫ 35/100 [00:06<00:11, 6it/s]
36.0%┣████████████████▏     

Performance stats (MQD loop only)
time [s]                    17.787434791
subwins/s              438.5117973248513
allocations [GB]                   12.94
allocations [MB/subwin]              165.92
time [m]                             0.3
total n subwindows                  7800
n image pairs                        100
n subwindows in FOV                   78
size(subwindows)                 (6, 13)
M, N                            (80, 80)
rhoI, rhoJ                      (80, 80)
allocations [B]              12941933097
allocations [MB]                12941.93
gc time [s]                  4.475822251
gc stats            Base.GC_Diff(12941933097, 39400, 0, 413369, 62, 30151, 4475822251, 4, 1)
number of threads                      8
Running on apple-m1 CPU specs:
Apple M1: 
       speed         user         nice          sys         idle          irq
#1  2400 MHz     267368 s          0 s     254784 s    1395296 s          0 s
#2  2400 MHz     246420 s          0 s     236387 s    14

In [87]:
# Fit streamlines to vector field

import PyPlot as plt
import DifferentialEquations
using Interpolations
import StaticArrays

function meshgrid(x::AbstractVector, y::AbstractVector)
    X = repeat(x', length(y), 1)
    Y = repeat(y, 1, length(x))
    return X, Y
end

# Ensure data is in Float64 format
i = Float64[y_vector[:,1]...]
j = Float64[x_vector[1,:]...]
U = float.(dropdims(Statistics.mean(subpix_shifts_j, dims = 3), dims = 3))
V = float.(dropdims(Statistics.mean(subpix_shifts_i, dims = 3), dims = 3))

# Check dimensions
println("Dimensions of i: ", length(i))
println("Dimensions of j: ", length(j))
println("Dimensions of U: ", size(U))
println("Dimensions of V: ", size(V))

# Create linear interpolants for the vector field
u_interp = extrapolate(interpolate((i, j), U, Gridded(Linear())), Line())
v_interp = extrapolate(interpolate((i, j), V, Gridded(Linear())), Line())

function vector_field(u, p, t)
    x, y = u
    U = u_interp(x, y)
    V = v_interp(x, y)
    return StaticArrays.SA[U, V]
end

# Define initial conditions and time span
initial_conditions = [(160, 450), (160, 550)]
initial_conditions = [StaticArrays.SA[sp[1], sp[2]] for sp in initial_conditions]

# Store coordinates of all streamlines
all_streamline_coords = []

# Termination condition function
function terminate_at_bounds(u, t, integrator)
    y, x = u[1], u[2] # y = row, x = col. Changed this from ChatGPT code (x, y and it fixed bounds issue)
    # Check if x or y are outside the bounds defined by Img_dims
    return x < 1 || x > Img_dims[2] || y < 1 || y > Img_dims[1]
end

# Termination action function
function terminate_action!(integrator)
    DifferentialEquations.terminate!(integrator)
end

# Compute streamline function
function compute_streamline(start_point)
    # Convert the start point to a StaticArray
    start_point_sa = StaticArrays.SA[start_point[1], start_point[2]]

    # Set time span for the ODE solver
    tspan = (0.0, 200.0)  # Adjust the time span as needed

    # Create the DiscreteCallback
    cb = DifferentialEquations.DiscreteCallback(terminate_at_bounds, terminate_action!)

    # Define the ODEProblem with the initial condition and vector field
    prob = DifferentialEquations.ODEProblem(vector_field, start_point_sa, tspan)

    # Solve the ODEProblem with the callback
    sol = DifferentialEquations.solve(prob, DifferentialEquations.Tsit5(), callback=cb)

    # Extract and return the streamline coordinates
    streamline_coords = hcat(sol.u...)'
    # return streamline_coords
    coords = [(sol[1, i], sol[2, i]) for i in 1:length(sol.t)]
    push!(all_streamline_coords, coords)
    print(streamline_coords)
end

# Compute streamlines for all starting points
for sp in initial_conditions
    compute_streamline(sp)
end

# Plot the vector field using quiver
X, Y = meshgrid(j, i)

fig, ax = plt.subplots()
ax.quiver(X, Y, U, V, angles="xy") # Why the ' on U and V?

# Plot all streamlines
for coords in all_streamline_coords
    x_coords = [coord[1] for coord in coords]
    y_coords = [coord[2] for coord in coords]
    ax.plot(x_coords, y_coords, label="Streamline")
end

# Set labels and title
ax.invert_yaxis()  # To match the coordinate system of the matrices
ax.set_title("Vector Field with Streamlines")
ax.legend()

# Show the plot
plt.show()

# Print the number of coordinates for each streamline
for (i, coords) in enumerate(all_streamline_coords)
    println("Streamline ", i, " has ", length(coords), " coordinates.")
end

Dimensions of i: 6
Dimensions of j: 13
Dimensions of U: (6, 13)
Dimensions of V: (6, 13)
[160.0 450.0; 161.8112079071258 449.4729117742668; 179.9697590304095 444.4403754131109; 265.05980621589686 425.94271439764526; 328.06395318055917 412.2733610862107; 409.6128624700594 394.3379339198215; 503.6645504698251 372.67712536547094; 623.9622563315247 344.55677318605973; 888.6837747799814 290.44972365906654; 888.6837747799814 290.44972365906654][160.0 550.0; 162.04918804463796 549.5039432626531; 182.63480370809654 544.7598094305158; 292.79153295128435 525.0200397444919; 390.51253358749204 507.6735704894758; 530.4539845383632 478.8917963699761; 694.4588221711002 437.68354666297915; 987.8721525733481 364.76604953536054; 987.8721525733481 364.76604953536054]Streamline 1 has 10 coordinates.
Streamline 2 has 9 coordinates.


In [92]:
# Find pixels under each streamline with 2 algorithms - Bresenham (single line) & Wu (anti-aliased)
StreamlinePixelsAntialiased = []
StreamlinePixelsSingle = []

for coords in all_streamline_coords
    StreamlinePixelsWu = []
    StreamlinePixelsBresenham = []
    for k in 1:length(coords)-1
        x0, y0 = coords[k]
        x1, y1 = coords[k+1]

        # Check for NaN values
        if isnan(x0) || isnan(y0) || isnan(x1) || isnan(y1)
            println("NaN detected in streamline coordinates, skipping this segment.")
            continue
        end

        # Check for zero-length segments
        if x0 == x1 && y0 == y1
            println("Zero-length segment detected, skipping this segment.")
            continue
        end

        # Process with Wu and Bresenham algorithms
        wu_pixels = wu_line(x0, y0, x1, y1)
        bresenham_pixels = bresenham_line(round(Int, x0), round(Int, y0), round(Int, x1), round(Int, y1))
        append!(StreamlinePixelsWu, wu_pixels)
        append!(StreamlinePixelsBresenham, bresenham_pixels)
    end
    push!(StreamlinePixelsAntialiased, StreamlinePixelsWu)
    push!(StreamlinePixelsSingle, StreamlinePixelsBresenham)
end

# Remove duplicate pixels from bresenham lines:
StreamlinePixelsSingle = [collect(Set(row)) for row in StreamlinePixelsSingle]

# # Plot the vector field using quiver
X, Y = meshgrid(j, i)

# Wu line plot:
fig, ax = plt.subplots()
ax.quiver(X, Y, U, V, angles="xy")

# Plot all streamlines
for coords in all_streamline_coords
    x_coords = [coord[1] for coord in coords]
    y_coords = [coord[2] for coord in coords]
    ax.plot(x_coords, y_coords, label="Streamline")
end

ax.invert_yaxis()  # To match the coordinate system of the matrices

# Visualize the pixels under each streamline
for streamline_pixels in StreamlinePixelsAntialiased
    for (x, y, c) in streamline_pixels
        ax.plot(x, y, "ro", markersize=2 * c, alpha=c, label="Streamline Pixels")
    end
end

# Set labels and title
# ax.invert_yaxis()  # To match the coordinate system of the matrices
ax.set_title("Vector Field with Streamlines and Antialiased Pixels")
# ax.legend()

plt.show()

# Bresenham line plot
fig, ax = plt.subplots()
ax.quiver(X, Y, U, V, angles="xy")

# Plot all streamlines
for coords in all_streamline_coords
    x_coords = [coord[1] for coord in coords]
    y_coords = [coord[2] for coord in coords]
    ax.plot(x_coords, y_coords, label="Streamline")
end

ax.invert_yaxis()  # To match the coordinate system of the matrices
# plt.show()

for streamline_pixels in StreamlinePixelsSingle
    for (x, y) in streamline_pixels
        ax.plot(x, y, "ro", markersize=2, label="Streamline Pixels")
    end
end

# Set labels and title
# ax.invert_yaxis()  # To match the coordinate system of the matrices
ax.set_title("Vector Field with Streamlines and Bresenham Pixels")
# ax.legend()

# Show the plot
plt.show()

Zero-length segment detected, skipping this segment.
Zero-length segment detected, skipping this segment.


In [90]:
test_MQD_pix = StreamlinePixelsSingle[2] # Long streamline from edge of image to just before entrance


829-element Vector{Any}:
 (963, 371)
 (687, 440)
 (423, 501)
 (245, 534)
 (744, 426)
 (823, 406)
 (487, 488)
 (568, 470)
 (428, 500)
 (522, 481)
 ⋮
 (883, 391)
 (173, 547)
 (406, 505)
 (809, 409)
 (980, 367)
 (779, 417)
 (552, 474)
 (853, 399)
 (384, 509)

In [80]:
# STIV test with MQD streamline
test_MQD_pix = StreamlinePixelsSingle[2] # Long streamline from edge of image to just before entrance
STIV_vec_MQD = zeros(length(test_MQD_pix), length(raw_imgs))

for img = 1:length(raw_imgs)
    for coord = 1:length(test_MQD_pix)
        j, i = test_MQD_pix[coord]
        STIV_vec_MQD[coord, img] = raw_imgs[i, j, img]
    end
end

plt.imshow(STIV_vec_MQD)
plt.show()


In [ ]:
import Pkg
Pkg.add("Dierckx")

In [11]:
# Polygon mask functions from https://github.com/JuliaImages/ImageDraw.jl/issues/56

function point_in_polygon(poly_xs::Vector{T}, poly_ys::Vector{T}, x::T, y::T) where T<: Real
    n_verts = length(poly_xs)
    j = n_verts
    c = false
    for i in 1:n_verts
        if (((poly_ys[i] <= y) && (y < poly_ys[j])) || ((poly_ys[j] <= y) && (y < poly_ys[i]))) && 
            (x < (poly_xs[j] - poly_xs[i]) * (y - poly_ys[i]) / (poly_ys[j] - poly_ys[i]) + poly_xs[i])
            c = !c
        end
        j = i
    end
    return c
end

# Changed these two functions to use bool matrix instead of values
function draw_polygon!(mask::Matrix{Bool}, poly_xs::Vector{Int}, poly_ys::Vector{Int})
    min_x, max_x = max(minimum(poly_xs), 1), min(maximum(poly_xs), size(mask, 2))
    min_y, max_y = max(minimum(poly_ys), 1), min(maximum(poly_ys), size(mask, 1))
    for y in min_y:max_y
        for x in min_x:max_x
            if point_in_polygon(poly_xs, poly_ys, x, y)
                mask[y, x] = true
            end
        end
    end
end

function polygons_to_mask(polygons::Array{Matrix{T}, 1} where T <: Real, max_x::Int, max_y::Int)::Matrix{Bool}
    poly_mask = zeros(Bool, max_y, max_x)  # Initialize a Bool matrix with all values set to false
    for p in polygons
        draw_polygon!(poly_mask, round.(Int, p[:, 1]), round.(Int, p[:, 2]))
    end
    return poly_mask
end


polygons_to_mask (generic function with 1 method)

In [9]:
# Testing auto STIV w/ real output
STIV_vec_Canny_sq = STIV_vec_Canny[251:749,1:499]
plt.imshow(STIV_vec_Canny_sq)
plt.show()
size(STIV_vec_Canny_sq)

(499, 499)

In [14]:
import Images
import LinearAlgebra
import FFTW
import PyPlot as plt  # Use plt for plotting

# Initialize variables
# y = zeros(Float64, length(x), length(x))
# y1 = zeros(Float64, length(x), length(x))
# y2 = zeros(Float64, length(x), length(x))

# Populate y1, y2, and y arrays
# for i in 1:length(x)
#     y1[i, :] = sin.(0.15 * (x .- i/2))
#     y2[i, :] = 0.9 .* sin.(0.05 * (x .- i))
#     y[i, :] = y1[i, :] .+ y2[i, :]
# end
y = STIV_vec_Canny_sq
x = 1:size(STIV_vec_Canny_sq)[1]
xCenter = 250
yCenter = 250
radius = 249

# Display the y array as an image
# plt.figure()
# plt.imshow(y)
# plt.title("y Array")
# plt.colorbar()

# Compute the 2D Fourier transform and the power spectrum
Rxy = FFTW.ifft(abs.(FFTW.fft(y)).^2)
plt.figure()
plt.imshow(real(log.(Rxy[2:end,:])))
plt.title("Rxy Power Spectrum")
plt.colorbar()

# Calculate absolute value of spectra to plot
Rxy_real = real(Rxy)

# # Create a polar array
Rxy_pol = zeros(Float64, length(x), length(x))

# Define center and radius for circular mask
# xCenter = 501
# yCenter = 501
# radius = 500


# Define angles and compute coordinates for the circle
# theta_c = range(0, stop=2*pi, length=round(Int, 4 * pi * radius))
θ_c = range(1, stop = 360, length = 360)
xc = radius * cosd.(θ_c) .+ xCenter
yc = radius * sind.(θ_c) .+ yCenter

# Create a circular mask
circ_mask = polygons_to_mask([hcat(xc, yc)], xCenter+radius, yCenter+radius)

circ_nan = ones(Float64, size(circ_mask))
circ_nan[.!circ_mask] .= NaN

θ = zeros(Float64, length(x), length(x))
ρ = zeros(Float64, length(x), length(x))
Tx = zeros(Float64, length(x), length(x))
Ty = zeros(Float64, length(x), length(x))
M = 15  # "coefficient of intensification"

# Calculate theta and ρ for each pixel within the mask
for i in 1:length(x)
    for j in 1:length(x)
        if circ_mask[i, j]
            Tx[i,j] = xCenter - j
            Ty[i,j] = yCenter - i
            θ[i,j] = atand(Ty[i,j] / Tx[i, j])
            ρ[i,j] = M * log(sqrt(Tx[i,j]^2 + Ty[i,j]^2))
        end
    end
end

# Apply the circular mask to Rxy
Rxy_pol_norm = Rxy_real .* circ_nan ./ Rxy_real[yCenter, xCenter]
plt.figure()
plt.imshow(Rxy_pol_norm)
plt.title("Rxy Polar")
plt.colorbar()

ρ = ρ .* circ_nan
θ = θ .* circ_nan

# plt.figure()
# plt.imshow(θ)
# plt.title("Theta")
# plt.colorbar()

# plt.figure()
# plt.imshow(ρ)
# plt.title("Rho")
# plt.colorbar()

# Convert theta to degrees and calculate polar coordinates
# theta_deg = theta .* (180 / pi) .+ 90  # make all values positive
pol_x = exp.(ρ) .* cos.(θ)
pol_y = exp.(ρ) .* sin.(θ)

# plt.figure()
# plt.pcolormesh(ρ, theta_deg, Rxy_pol)
# plt.colorbar()
# plt.title("Polar Mesh")

# # Calculate the rounded rho array

# Aggregate by polar coordinates
# AngleIncrement = 0.1
AngleVec = [1*AngleIncrement:1*AngleIncrement:180]
θ_int = zeros(Float64, AngleVec)
inds_vec = zeros(Int, AngleVec)

for i in 1:AngleVec
    a = i-90
    inds = findall(θ -> round(θ) == a, θ[:])
    inds_vec[i] = length(inds)
    θ_int[i] = sum(Rxy_pol_norm[inds])
end

# plt.figure()
# plt.plot(inds_vec)
# plt.title("Indices Vector")

# plt.figure()
# plt.plot(θ_int)
# plt.title("Theta Integral")

# Rxy_norm = Rxy[xCenter, yCenter]
# θ_int_norm = θ_int ./ (Rxy_norm * Statistics.mean(inds_vec))

# plt.figure()
# plt.plot(θ_int_norm)
# plt.title("Normalized Theta Integral")


ρ_round = round.(ρ)
μ = zeros(Float64, degs)
ρ_max = M*log(minimum([maximum(filter(!isnan,Tx)),maximum(filter(!isnan,Ty))]))
for i in 1:degs
    a = i-90
    inds = findall(θ -> round(θ, digits = 1) == a, θ[:])
    Rxy_int = sum(Rxy_pol_norm[inds])
    μ[i] = (1/ρ_max)*Rxy_int
end

plt.figure()
plt.plot(μ)
plt.title("Directional Average Distribution")
plt.show()


MethodError: MethodError: no method matching zeros(::Type{Float64}, ::Vector{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}})

Closest candidates are:
  zeros(::Type{T}, !Matched::Tuple{}) where T
   @ Base array.jl:588
  zeros(::Type{T}, !Matched::Tuple{StaticArrays.SOneTo, Vararg{StaticArrays.SOneTo}}) where T
   @ StaticArrays ~/.julia/packages/StaticArrays/MSJcA/src/abstractarray.jl:299
  zeros(::Type{T}, !Matched::Tuple{Vararg{Integer, N}}) where {T, N}
   @ Base array.jl:583
  ...


In [86]:
[maximum(filter(!isnan,Tx)),maximum(filter(!isnan,Ty))]

2-element Vector{Float64}:
 249.0
 249.0

In [81]:
maximum(filter(!isnan, ρ))
ρ_round = round.(ρ)
ρ_round[200,200]
maximum(filter(!isnan, ρ_round))

83.0

In [72]:
plt.figure()
plt.pcolormesh(ρ, theta_deg, Rxy_pol)
plt.colorbar()
plt.title("Polar Mesh")

UndefVarError: UndefVarError: `theta_deg` not defined

In [24]:
θ_int

180-element Vector{Float64}:
  9.969514657364172e7
  1.4428202792270762e8
  5.6719990876102135e7
  3.825093297192533e7
  1.1395192063056597e8
  1.5613178181381285e8
  7.267214232152568e7
 -6.957507119521332e7
 -1.4151729655851343e8
 -1.030216146148636e8
  ⋮
  2.1733744785245162e8
  1.167973715753948e7
 -1.9583936519642577e8
 -1.8364465394977325e8
 -5.981483475526104e7
 -6.779630610958755e7
 -1.7346765732381043e8
 -2.0392009551065218e8
  1.185459163688675e7

In [107]:
deg_start = 1
deg_end = 180

for m in deg_start:deg_end
    # print(m)
    # inds = findall(θ -> round(Int, θ) == m, θ[:])
    # inds_vec[m] = length(inds)
    # θ_int[m] = sum(Rxy_pol[inds])
end
degs = 180
θ_int = zeros(Float64, degs)
# inds_vec = zeros(Int, degs)

180-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [69]:
plt.figure()
# plt.imshow(round.(θ, digits = 0))
plt.imshow(Ty)
plt.colorbar()
plt.show()


In [67]:
Tx

1001×1001 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱       ⋮                        ⋮
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [51]:
theta_c = range(1, stop = 360, length = 360)
xc = radius * cosd.(theta_c) .+ xCenter
yc = radius * sind.(theta_c) .+ yCenter

# Create a circular mask
circ_mask = zeros(Bool, 1001, 1001)
for i in 1:length(theta_c)
    circ_mask[round(Int, yc[i]), round(Int, xc[i])] = true
end



function point_in_polygon(poly_xs::Vector{T}, poly_ys::Vector{T}, x::T, y::T) where T<: Real
    n_verts = length(poly_xs)
    j = n_verts
    c = false
    for i in 1:n_verts
        if (((poly_ys[i] <= y) && (y < poly_ys[j])) || ((poly_ys[j] <= y) && (y < poly_ys[i]))) && 
            (x < (poly_xs[j] - poly_xs[i]) * (y - poly_ys[i]) / (poly_ys[j] - poly_ys[i]) + poly_xs[i])
            c = !c
        end
        j = i
    end
    return c
end

function draw_polygon!(mask::Matrix{T2}, poly_xs::Vector{T}, poly_ys::Vector{T}, value::T2) where T<: Integer where T2 <: Real
    min_x, max_x = max(minimum(poly_xs), 1), min(maximum(poly_xs), size(mask, 2))
    min_y, max_y = max(minimum(poly_ys), 1), min(maximum(poly_ys), size(mask, 1))
    for y in min_y:max_y
        for x in min_x:max_x
            if point_in_polygon(poly_xs, poly_ys, x, y)
                mask[y, x] = value
            end
        end
    end
end

function polygons_to_mask(polygons::Array{Matrix{T}, 1} where T <: Real, max_x::Int, max_y::Int)
    poly_mask = zeros(Int, max_y, max_x);
    for (i,p) in enumerate(polygons)
        draw_polygon!(poly_mask, round.(Int, p[:,1]), round.(Int, p[:,2]), i)
    end
    return poly_mask
end

circ_mask = polygons_to_mask([hcat(xc, yc)], 1001, 1001)


1001×1001 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 ⋮              ⋮              ⋮        ⋱     ⋮              ⋮              ⋮
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0 